In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

auth = r"D:\DocumentsSte\VSCode\Proj1Streamlit\resources\car_prices.csv"

df = pd.read_csv(auth)

df = df.dropna()
df = df.drop_duplicates()

In [ ]:
df_make_value = df.groupby(["make"])[['sellingprice']].agg(['mean','sum','std'])

df_make_volume = df.groupby(["make"])[['vin']].agg(['count'])

top_brand_value = df_make_value['sellingprice']['sum'].idxmax()
top_sales_value = df_make_value['sellingprice']['sum'].max()

top_brand_volume = df_make_volume['vin']['count'].idxmax()
top_sales_volume = df_make_volume['vin']['count'].max()

print(f"Highest selling brand: {top_brand_value} with total sales: ${top_sales_value:,.2f} and total sold vehicles: {top_sales_volume:,.2f}")



Highest selling brand: Ford with total sales: $1,201,407,350.00 and total sold vehicles: 81,013.00
Highest selling brand: Ford with total sales: 81,013.00


In [19]:
df_make_value = df.groupby(["make","model"])[['sellingprice']].agg(['mean','sum','std'])

df_make_volume = df.groupby(["make","model"])[['vin']].agg(['count'])

top_brand_value = df_make_value['sellingprice']['sum'].idxmax()
top_sales_value = df_make_value['sellingprice']['sum'].max()

top_brand_volume = df_make_volume['vin']['count'].idxmax()
top_sales_volume = df_make_volume['vin']['count'].max()

print(f"Highest selling brand/model: {top_brand_value} with total sales: ${top_sales_value:,.2f}")

print(f"Highest selling brand/model: {top_brand_volume} with total sold vehicles: {top_sales_volume:,.2f}")

Highest selling brand/model: ('Ford', 'F-150') with total sales: $234,075,458.00
Highest selling brand/model: ('Nissan', 'Altima') with total sold vehicles: 16,346.00


In [ ]:
# group by body and make and get the sum of sellingprice
df_body_make_value = df.groupby(["body","make"])[['sellingprice']].agg(['sum'])

# get the id of the max selling price for each body type
# apply(lambda x: x[1]) gets a element of a list or tuple in case of x[0] is the first element of the tuple
max_sales_per_body1 = df_body_make_value['sellingprice']['sum'].groupby("body").idxmax().apply(lambda x: x[1])

# get the max selling price for each body type
max_sales_per_body2 = df_body_make_value['sellingprice']['sum'].groupby("body").max()

max_sales_per_body = pd.merge(max_sales_per_body1, max_sales_per_body2, left_index=True, right_index=True)

#change name of columns
max_sales_per_body.rename(columns={'sum_x': 'Make','sum_y': 'Total_SellingPrice'}, inplace=True)

max_sales_per_body

,Make,Total_SellingPrice
body,,
Access Cab,Toyota,2898100.0
Beetle Convertible,Volkswagen,828500.0
CTS Coupe,Cadillac,2303350.0
CTS Wagon,Cadillac,184850.0
CTS-V Coupe,Cadillac,752371.0
...,...,...
transit van,Ford,158950.0
tsx sport wagon,Acura,106650.0
van,GMC,2333200.0


In [ ]:
# group by body, make and model and get the sum of sellingprice
df_body_make_value = df.groupby(["body","make","model"])[['sellingprice']].agg(['sum'])

# get the id of the max selling price for each body type
# apply(lambda x: x[1]) gets a element of a list or tuple in case of x[0] is the first element of the tuple
max_sales_per_body1 = df_body_make_value['sellingprice']['sum'].groupby("body").idxmax().apply(lambda x: x[1]+'/'+x[2])

# get the max selling price for each body type
max_sales_per_body2 = df_body_make_value['sellingprice']['sum'].groupby("body").max()

max_sales_per_body = pd.merge(max_sales_per_body1, max_sales_per_body2, left_index=True, right_index=True)

#change name of columns
max_sales_per_body.rename(columns={'sum_x': 'Model','sum_y': 'Total_SellingPrice'}, inplace=True)

max_sales_per_body

,Model,Total_SellingPrice
body,,
Access Cab,Toyota/Tacoma,2222200.0
Beetle Convertible,Volkswagen/Beetle Convertible,828500.0
CTS Coupe,Cadillac/CTS Coupe,2303350.0
CTS Wagon,Cadillac/CTS Wagon,184850.0
CTS-V Coupe,Cadillac/CTS-V Coupe,752371.0
...,...,...
transit van,Ford/Transit Van,158950.0
tsx sport wagon,Acura/TSX Sport Wagon,106650.0
van,GMC/Savana Cargo,2301200.0


: 

In [12]:
make_filtered = {'Volvo','BMW'}

# Filter DataFrame if at least one category is selected
if make_filtered:
    filtered_df = df[df['make'].isin(make_filtered)]
else:
    filtered_df = df  # Show all data if no selection

filtered_df

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
2,2014,BMW,3 Series,328i SULEV,Sedan,automatic,wba3c1c51ek116351,ca,45.0,1331.0,gray,black,financial services remarketing (lease),31900.0,30000.0,Thu Jan 15 2015 04:30:00 GMT-0800 (PST)
3,2015,Volvo,S60,T5,Sedan,automatic,yv1612tb4f1310987,ca,41.0,14282.0,white,black,volvo na rep/world omni,27500.0,27750.0,Thu Jan 29 2015 04:30:00 GMT-0800 (PST)
4,2014,BMW,6 Series Gran Coupe,650i,Sedan,automatic,wba6b2c57ed129731,ca,43.0,2641.0,gray,black,financial services remarketing (lease),66000.0,67000.0,Thu Dec 18 2014 12:30:00 GMT-0800 (PST)
6,2014,BMW,M5,Base,Sedan,automatic,wbsfv9c51ed593089,ca,34.0,14943.0,black,black,the hertz corporation,69000.0,65000.0,Wed Dec 17 2014 12:30:00 GMT-0800 (PST)
19,2014,BMW,6 Series,650i,Convertible,automatic,wbayp9c53ed169260,ca,34.0,8819.0,black,black,the hertz corporation,68000.0,67200.0,Wed Dec 17 2014 12:30:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
558783,2003,BMW,3 Series,330i,Sedan,automatic,wbaev53413km28443,ca,37.0,124583.0,silver,—,central auto sales inc,3425.0,4500.0,Wed Jul 08 2015 09:05:00 GMT-0700 (PDT)
558804,2007,Volvo,XC90,3.2,suv,automatic,yv4cy982971355864,nv,34.0,95769.0,white,beige,unique autos,5950.0,8500.0,Thu Jul 09 2015 09:00:00 GMT-0700 (PDT)
558808,2008,BMW,1 Series,135i,Convertible,automatic,wbaun93588vf56134,ca,33.0,96344.0,red,black,autonet trading,12500.0,13000.0,Wed Jul 08 2015 09:30:00 GMT-0700 (PDT)
558831,2011,BMW,5 Series,528i,Sedan,automatic,wbafr1c53bc744672,fl,39.0,66403.0,white,brown,lauderdale imports ltd bmw pembrok pines,20300.0,22800.0,Tue Jul 07 2015 06:15:00 GMT-0700 (PDT)


In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from config.auth import auth as auth

data = pd.read_csv(auth)

data

ModuleNotFoundError: No module named 'config'